In [1]:
import pandas as pd
import plotly.express as px

In [2]:
dfRaw = pd.read_excel('annual_generation_state.xls', skiprows=1)
dfRaw.head(3)

,YEAR,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours)
0,1990,AK,Total Electric Power Industry,Total,5599506.0
1,1990,AK,Total Electric Power Industry,Coal,510573.0
2,1990,AK,Total Electric Power Industry,Hydroelectric Conventional,974521.0


In [6]:
# filtering data only looking at total production instead of for each producer type
dfFilter = dfRaw[dfRaw['TYPE OF PRODUCER'] == 'Total Electric Power Industry'].copy()
del dfFilter['TYPE OF PRODUCER']

# defining the sources that I want to look at in this project, then removing other sources
SourceList = ['Total', 
              'Coal', 
              'Hydroelectric Conventional',
              'Natural Gas', 
              'Wind', 
              'Nuclear', 
              'Solar Thermal and Photovoltaic'
              ]
dfFilter = dfFilter[dfFilter['ENERGY SOURCE'].isin(SourceList)]

# Renaming a column
dfFilter.rename(columns={'GENERATION (Megawatthours)' : 'MWh'}, inplace=1)

# updating an inconsistency in capitalization of some values
dfFilter['STATE'].replace({'US-Total': 'US-TOTAL'}, inplace=True)

# Creating a new column based on the total for each/year state
dfTotals = dfFilter[dfFilter['ENERGY SOURCE'] == 'Total'].copy()
dfTotals.rename(columns={'MWh': 'Total MWh'}, inplace=True)
del dfTotals['ENERGY SOURCE']
dfMerged = pd.merge(dfTotals, dfFilter, on=['YEAR', 'STATE'])

# using the new column to find the percentage of totals for each energy source per year and state
dfMerged['Percent'] = dfMerged['MWh']/dfMerged['Total MWh']*100

# deleting total column since it is no longer needed
del dfMerged['Total MWh']

dfMerged.to_csv('CleanData.csv')
dfMerged.head(3)

C:\Users\seana3\AppData\Local\Temp\ipykernel_17856\1316423654.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfFilter['STATE'].replace({'US-Total': 'US-TOTAL'}, inplace=True)
C:\Users\seana3\AppData\Local\Temp\ipykernel_17856\1316423654.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTotals.rename(columns={'MWh': 'Total MWh'}, inp

,YEAR,STATE,ENERGY SOURCE,MWh,Percent
0,1990,AK,Total,5599506.0,100.000000
1,1990,AK,Coal,510573.0,9.118179
2,1990,AK,Hydroelectric Conventional,974521.0,17.403696
